In [ ]:
# General Imports
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import os
import kagglehub
import cv2


# Visualization
import matplotlib.pyplot as plt

# Building Model
from keras.utils import plot_model
from tensorflow.keras import models
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.optimizers import Adam

# Training Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

# Data Processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [65]:
#Download data set and get the path
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path);


Path to dataset files: /Users/nehiraltinkaya/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1


In [66]:
train_dir = "/Users/nehiraltinkaya/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training"
test_dir = "/Users/nehiraltinkaya/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Testing"
#train_dataset = train_dir.flow_from_directory(train_dir, target_size=(250,250))
#test_dataset = test_dir.flow_from_directory(test_dir, target_size=(250,250))


In [67]:
cv2.imread("/Users/nehiraltinkaya/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training/glioma/Tr-glTr_0000.jpg").shape

(512, 512, 3)

In [68]:
img = load_img("/Users/nehiraltinkaya/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training/glioma/Tr-glTr_0000.jpg")

In [69]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    batch_size=32,
    label_mode="categorical",
    image_size=(256, 256),
    color_mode= 'rgb'
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    batch_size=32,
    label_mode="categorical",
    image_size=(256, 256),
    color_mode= 'rgb'
)


Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [70]:
train_dataset.class_names

['glioma', 'meningioma', 'notumor', 'pituitary']

In [71]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(256,256,3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Dropout 25% of the nodes of the previous layer during training
model.add(Flatten())     # Flatten, and add a fully connected layer
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(train_dataset, epochs=15, batch_size=32, validation_data=test_dataset) 


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 254, 254, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 252, 252, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 508032)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │    65,028,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,033,828 (248.08 MB)

 Trainable params: 65,033,828 (248.08 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.3769 - loss: 369.5688 - val_accuracy: 0.5278 - val_loss: 1.1232
Epoch 2/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.5143 - loss: 1.1467 - val_accuracy: 0.7269 - val_loss: 0.8968
Epoch 3/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.5949 - loss: 0.9570 - val_accuracy: 0.7292 - val_loss: 0.7583
Epoch 4/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.6245 - loss: 0.8630 - val_accuracy: 0.7864 - val_loss: 0.6259
Epoch 5/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 176s 981ms/step - accuracy: 0.6541 - loss: 0.7937 - val_accuracy: 0.7971 - val_loss: 0.5802
Epoch 6/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.6737 - loss: 0.7358 - val_accuracy: 0.8047 - val_loss: 0.5789
Epoch 7/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.6914 - loss: 0.6831 - val_accuracy: 0.8230 - val_loss: 0.5006
Epoch 8/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 178s 995ms/step - accuracy: 0.6978 - loss: 0.6689 - 

In [24]:
#for getting class indices
datagen = ImageDataGenerator(rescale=1./255)
train_dataset = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)
print(train_dataset.class_indices)
print(train_dataset.classes)

Found 5712 images belonging to 4 classes.
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
[0 0 0 ... 3 3 3]


In [ ]:
#utilizing cnn with maxpooling, and all the layers are given in a form of list, one by one we provide the layers, 
# so Conv2D(how many filters you want to keep inside this layer which is how many features to learn from, 
# (size of the filters), activation function, input_shape=(256, 256, 3-->dimension because rgb) )
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
                                    #then adding a max pooling layer
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    #4 because 4 classes
                                    #softmax because we have multiple classes and not binary
                                    tf.keras.layers.Dense(4, activation='softmax')])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,088,452 (49.93 MB)

 Trainable params: 13,088,452 (49.93 MB)

 Non-trainable params: 0 (0.00 B)

In [95]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [94]:
model_fit = model.fit(train_dataset,
          steps_per_epoch=50,  
          epochs=10,
          validation_data=test_dataset) 

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.7693 - loss: 0.5043 - val_accuracy: 0.8802 - val_loss: 0.3645
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.8089 - loss: 0.5283 - val_accuracy: 0.8574 - val_loss: 0.4021
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.8002 - loss: 0.4375 - val_accuracy: 0.8726 - val_loss: 0.3726
Epoch 4/10
29/50 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7958 - loss: 0.4522

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 741ms/step - accuracy: 0.7978 - loss: 0.4529 - val_accuracy: 0.8902 - val_loss: 0.4042
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.8040 - loss: 0.4341 - val_accuracy: 0.8711 - val_loss: 0.4503
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.7845 - loss: 0.4747 - val_accuracy: 0.8688 - val_loss: 0.4091
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.8163 - loss: 0.4548 - val_accuracy: 0.8894 - val_loss: 0.3579
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 755ms/step - accuracy: 0.8147 - loss: 0.4887 - val_accuracy: 0.8894 - val_loss: 0.4024
Epoch 9/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.7864 - loss: 0.5027 - val_accuracy: 0.8841 - val_loss: 0.3544
Epoch 10/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.8176 - loss: 0.4337 - val_accuracy: 0.9008 - val_loss: 0.4149
